In [122]:
#La probabilidad de que la versión actual de un contenido fuera editada sin dejar comentario para usuarios que están logueados y que no están logueados (⭐)

In [123]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 2s (182 kB/s)
Reading package lists... Done
Building dependency tree       
Reading s

In [124]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

from pyspark.sql.functions import isnan, when, count, col


In [125]:
from google.colab import drive
drive.mount('/content/drive')
#df_logs = pd.read_csv("/content/drive/MyDrive/Organizacion De Datos/Trabajo_practico/logs.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [127]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('/content/drive/MyDrive/Organizacion De Datos/Trabajo_practico/contents.csv', header=True, inferSchema=True)
rdd = df.rdd

In [128]:
rdd.count()

4132633

In [129]:
rdd.take(5)

[Row(title='Wikipedia:Artículos solicitados', id='5', namespace='4', revision_id='132533307', parent_revision='132516631', revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id='1708233', revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id='7', namespace='0', revision_id='138035057', parent_revision='138027475', revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido'),
 Row(title='Argentina', id='10', namespace='0', revision_id='137966826', parent_revision='137825165', revision_timestamp='2021-08-28T17:17:54Z', revisor_username='Traitrich', revisor_id='4642283', revisor_ip=None, revisor_comment='Reemplazos con [[Usuario:Benjavalero/Replacer|Replacer]]: «simbolos»'),
 Row(title='Geografía de Andorra', id='15', namespace='0', revision_id='137986788', parent_revision='133666383', revision_timestamp='2021-08-29T17:16:00Z',

In [130]:
rdd_sin_comentarios = rdd.filter( lambda x: (x.revisor_comment == None) or (x.revisor_comment == "") ) # Total de ediciones sin comentarios

In [131]:
rdd_sin_comentarios.count() # sin comentarios

469744

In [132]:
sin_coment_registrados = (rdd_sin_comentarios.filter(lambda x: x.revisor_id != None)).count()  # sin comentario y usuarios registrados 

In [133]:
sin_coment_no_registrados = (rdd_sin_comentarios.filter( lambda x: x.revisor_id == None or (x.revisor_id == "") )).count() # sin comentario y usuarios NO registrados

In [134]:
(100*sin_coment_registrados) / rdd.count()

9.490317673986535

In [135]:
(100*sin_coment_no_registrados) / rdd.count()

1.8763824418960018